In [1]:
import matplotlib
matplotlib.use('Agg')
import csv
from dataclasses import dataclass, field, asdict, astuple
from typing import List
from collections import namedtuple
from functools import reduce
from functools import partial
# import forallpeople as si ## 단위 변환 관련 패키지(아직 미사용)
import math
# %matplotlib Tk

import matplotlib.pyplot as plt
import numpy as np
import datetime
# import matplotlib.transforms as transforms

In [2]:
def importCSV(_path):
    with open(_path,'r') as f:
        reader = csv.reader(f, delimiter='\t')
        res = [x for x in reader]
        
        new_list = []
        for sublist in res:
            # 첫 항의 띄어쓰기 수 계산
            num_spaces = len(sublist[0]) - len(sublist[0].lstrip())

            # 띄어쓰기를 제거하여 새로운 리스트에 저장
            new_sublist = [sublist[0].lstrip()]
            new_sublist.extend([item.strip() for item in sublist[1:]])

            # 첫 항의 띄어쓰기 수를 첫 요소로 추가하여 저장
            new_sublist.insert(0, num_spaces)
            new_list.append(new_sublist)
            # 코드 앞4글자를 추가하여 저장
            new_sublist.insert(1,sublist[0].lstrip()[7:9])
        
            new_list.append(new_sublist)
        
    return new_list

In [3]:
def exportCSV(_filename, _datas):
    f = open(_filename, 'w', newline='') # 자동줄바꿈 방지 header 이후 첫행 빈행 생성 방지
    csv_writer = csv.writer(f)
    
    for x in _datas:
        csv_writer.writerow(x)
    f.close()

In [4]:
#################################Module For Functional Programing#############################################
curry = lambda f: lambda a,*args: f(a, *args) if (len(args)) else lambda *args: f(a, *args)

filter = curry(filter)
map = curry(map)

def _take(length, iter):
    res = []
    for a in iter:
        res.append(a)
        if len(res) == length:
            return res
        
take = curry(_take)
reduce = curry(reduce)

go = lambda *args: reduce(lambda a,f: f(a), args) ## 함수도 축약 가능 ##
##############################################################################################################

In [5]:
import os
os.getcwd()

'/home/jovyan/notebooks/documentum/건축심화세미나/20230329_hjs문의/230327 간트차트'

In [6]:
input_path = './Schedule Input.csv'

In [7]:
dfScheduleSet = importCSV(input_path)
dfScheduleSet

[[0,
  'y ',
  'Activity ID',
  '(New)',
  'Activity Name',
  'Duration',
  'Start',
  'Finish'],
 [0,
  'y ',
  'Activity ID',
  '(New)',
  'Activity Name',
  'Duration',
  'Start',
  'Finish'],
 [0,
  'HA',
  'S-OIL SHAHEEN PROJECT_E & P SCHEDULE_작업중',
  '',
  '',
  '1007',
  '02-Jan-23',
  '04-Oct-25'],
 [0,
  'HA',
  'S-OIL SHAHEEN PROJECT_E & P SCHEDULE_작업중',
  '',
  '',
  '1007',
  '02-Jan-23',
  '04-Oct-25'],
 [2,
  ' #',
  'Package #1 - SCC / UTOS',
  '',
  '',
  '1007',
  '02-Jan-23',
  '04-Oct-25'],
 [2,
  ' #',
  'Package #1 - SCC / UTOS',
  '',
  '',
  '1007',
  '02-Jan-23',
  '04-Oct-25'],
 [4,
  'ri',
  'Engineering - PKG-1 작성 중',
  '',
  '',
  '545',
  '02-Jan-23',
  '20-Mar-25'],
 [4,
  'ri',
  'Engineering - PKG-1 작성 중',
  '',
  '',
  '545',
  '02-Jan-23',
  '20-Mar-25'],
 [6, '- ', 'Common - PKG-1 작성 중', '', '', '545', '02-Jan-23', '20-Mar-25'],
 [6, '- ', 'Common - PKG-1 작성 중', '', '', '545', '02-Jan-23', '20-Mar-25'],
 [8, ' M', 'Project Management', '', '', '62', '

In [8]:
dfHeader = dfScheduleSet.pop(0)
dfHeader

[0,
 'y ',
 'Activity ID',
 '(New)',
 'Activity Name',
 'Duration',
 'Start',
 'Finish']

In [18]:
def makeScheduleForm(args):
    
    Form = namedtuple('ScheduleForm', ['Level','Cate','ActivityID', 'New', 'ActivityName', 'Duration', 'Start', 'Finish']
                     )
    form =         Form(Level=args[0], Cate=args[1], ActivityID=args[2], New=args[3], 
            ActivityName=args[4], Duration=args[5], 
            Start=args[6], Finish=args[7])        
    
    return form

_targetSchedule = list(filter(lambda x: makeScheduleForm(x).Cate== "AR", dfScheduleSet))[0]
targetSchedules = makeScheduleForm(_targetSchedule)
targetSchedules

ScheduleForm(Level=12, Cate='AR', ActivityID='1SES201AR0010A', New='1', ActivityName='Architectural Basic Drawing for Main & Hot Section Substation (SS #90 & #91)_IFA', Duration='50', Start='15-Dec-23', Finish='28-Feb-24')

In [19]:
## 여기서 makeSectionForms는 데이터를 저장하지 않고 함수 객체를 저장한다. C#, JAVA에서의 Delegate와 동일
makeScheduleForms = lambda _list: go\
(
    _list,
    map(makeScheduleForm),
    filter(lambda x: makeScheduleForm(x).Cate =="AR"),
    
list)

In [20]:
multiScheduleForm = lambda _list: go\
(
    _list,
    map(makeScheduleForm),
    filter(lambda x: makeScheduleForm(x).Cate =="AR"),
    
list)

In [21]:
multiScheduleForm(targetSchedules)

TypeError: 'int' object is not subscriptable

In [22]:
import matplotlib.pyplot as plt

def draw_gantt_chart(schedule_forms):
    # 시작일, 종료일, 활동명, 활동번호 추출
    starts = [datetime.datetime.strptime(schedule.Start, "%d-%b-%y") for schedule in schedule_forms]
    ends = [datetime.datetime.strptime(schedule.Finish, "%d-%b-%y") for schedule in schedule_forms]
    activities = [schedule.ActivityName for schedule in schedule_forms]
    ids = [schedule.ActivityID for schedule in schedule_forms]

    sorted_schedules = sorted(schedule_forms, key=lambda x: x.Start)
    fig, axs = plt.subplots(len(set(schedule.Cate for schedule in schedule_forms)), figsize=(10, 5), sharey=True)
    
    axs = np.array(axs).flatten() # axs를 1차원 리스트로 변환
    
    for i, category in enumerate(set(schedule.Cate for schedule in schedule_forms)):
        category_schedules = list(filter(lambda x: x.Cate == category, sorted_schedules))
        bars = axs[i].barh(range(len(category_schedules)), 
                           [(schedule.Finish - schedule.Start).days for schedule in category_schedules],
                           left=[(schedule.Start - sorted_schedules[0].Start).days for schedule in category_schedules],
                           height=0.5, align='center', color='green', alpha=0.5)
        
        axs[i].set_title(category)
        axs[i].set_yticks(range(len(category_schedules)))
        axs[i].set_yticklabels([schedule.ActivityName for schedule in category_schedules])
        axs[i].set_xlim(0, (sorted_schedules[-1].Finish - sorted_schedules[0].Start).days + 1)
        axs[i].invert_yaxis()

    plt.tight_layout()
    plt.show()

In [23]:
import matplotlib.pyplot as plt

def draw_gantt_chart(schedule_forms):
    # 시작일, 종료일, 활동명, 활동번호 추출
    starts = [datetime.datetime.strptime(form.Start, "%d-%b-%y") for form in schedule_forms]
    ends = [datetime.datetime.strptime(form.Finish, "%d-%b-%y") for form in schedule_forms]
    activities = [form.ActivityName for form in schedule_forms]
    ids = [form.ActivityID for form in schedule_forms]

    # 그래프 설정
    fig, ax = plt.subplots(figsize=(10, 60))
    ax.grid(True)
    ax.set_ylim([0, 1])
    ax.set_xlim([min(starts), max(ends)])
    ax.set_xlabel('Date')
    ax.set_yticks([0.5])
    ax.set_yticklabels(['Activities'])

    # 각각의 활동을 그리기 위한 반복문
    for i in range(len(schedule_forms)):
        ax.broken_barh([(starts[i], (ends[i] - starts[i]).days)], (0.25, 0.5), edgecolors='black')
        ax.text(starts[i], 0.5, activities[i] + " (" + ids[i] + ")", ha='left', va='center')

    plt.show()

In [24]:
len(multiScheduleForm(targetSchedules))

TypeError: 'int' object is not subscriptable

In [25]:
def create_gantt_chart(schedule_list):
    activity_times = {}
    for schedule in schedule_list:
        activity_times[schedule.ActivityName] = (datetime.datetime.strptime(schedule.Start, '%d-%b-%y'), 
                                                  datetime.datetime.strptime(schedule.Finish, '%d-%b-%y'))
    
    fig, ax = plt.subplots(figsize=(50, len(schedule_list)/6))
    
    # Set font size
    plt.rcParams.update({'font.size': 5})
    
     # Set x-axis range
    min_date = min(activity_times.values(), key=lambda x: x[0])[0] - datetime.timedelta(days=30)
    max_date = max(activity_times.values(), key=lambda x: x[1])[1] + datetime.timedelta(days=30)
    plt.xlim(min_date, max_date)
    
    # Plot horizontal bars for each activity
    y_pos = list(range(len(activity_times)))
    height = 0.4
    for idx, (activity, times) in enumerate(activity_times.items()):
        ax.broken_barh([(times[0], times[1]-times[0])], (idx+height/4, height/2), 
                       facecolors=['tab:blue'])
        ax.text(times[0] + (times[1]-times[0])/2, idx+height/2, activity, ha='center', va='center')
    
    # Set y-axis labels
    ax.set_yticks(y_pos)
    ax.set_yticklabels(activity_times.keys())
    
    # Set x-axis label
    ax.set_xlabel('Date')
    
    # Set plot title
    ax.set_title('Gantt Chart')
    
    # Show plot
    plt.show()

In [26]:
create_gantt_chart(multiScheduleForm(targetSchedules))

TypeError: 'int' object is not subscriptable